## libs

In [2]:
# python utils
import itertools
import inspect
import random

# pre-processing and exploring data
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import statistics

# viz
import matplotlib.pyplot as plt
font = {'size'   : 15}
plt.rc('font', **font)

# model building
from pyod.models.auto_encoder import AutoEncoder
from keras.losses import mean_squared_error

Using TensorFlow backend.
/databricks/python/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping, MutableMapping

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:203: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

Out[2]: ['/job:localhost/replica:0/task:0/device:GPU:0']

## Import Data

In [5]:
table = "datalake_amplitude_clean_staging.`170698_listing_page_viewed_events`"
metrics_table_name = ("{}_daily_metrics".format(table.replace(".", "_").replace("`", "")))
enriched_metrics_table_name = metrics_table_name + "_enriched"
metrics_db = "dataops_anomaly_metrics"
spark_df = sqlContext.table("{}.`{}`".format(metrics_db, enriched_metrics_table_name))
display(spark_df)

n_rows,moving_avg_7d_n_rows,null_values_ratio,moving_avg_7d_null_values_ratio,max_string_length,moving_avg_7d_max_string_length,min_string_length,moving_avg_7d_min_string_length,avg_string_length,moving_avg_7d_avg_string_length,max_numeric,moving_avg_7d_max_numeric,min_numeric,moving_avg_7d_min_numeric,avg_numeric,moving_avg_7d_avg_numeric,date
40483.0,40483.0,0.40906398166124963,0.40906398166124963,2894.0,2894.0,1.0,1.0,35.460970851471856,35.460970851471856,1.515413428144E12,1.515413428144E12,1.0,1.0,1.9400959367022818E11,1.9400959367022818E11,2018-01-01
101110.0,70796.5,0.4171683264070417,0.4131161540341457,2902.0,2898.0,1.0,1.0,35.62608796807974,35.5435294097758,1.517500687669E12,1.5164570579065E12,1.0,1.0,1.9405875476468732E11,1.9403417421745776E11,2018-01-02
121895.0,87829.33333333333,0.41651931472515374,0.4142505409311484,2894.0,2896.6666666666665,1.0,1.0,35.72991149682971,35.605656772127105,1.515765249688E12,1.516226455167E12,1.0,1.0,1.9408343384345404E11,1.940505940927899E11,2018-01-03
115572.0,94765.0,0.4178751992069753,0.41515670550010514,3152.0,2960.5,1.0,1.0,35.645095876937816,35.61551654832978,1.515268023692E12,1.51598684729825E12,1.0,1.0,1.9409295957810974E11,1.9406118546411984E11,2018-01-04
106311.0,97074.2,0.4168528064909933,0.41549592569828275,3149.0,2998.2,1.0,1.0,35.85567009352586,35.663547257369,1.519491276563E12,1.5166877331512E12,1.0,1.0,1.9411301605076523E11,1.940715515814489E11,2018-01-05
84731.0,95017.0,0.40637220338893026,0.4139753053133907,2844.0,2972.5,1.0,1.0,34.976110328922275,35.54897443596121,1.516476968114E12,1.516652605645E12,1.0,1.0,1.9411862837261276E11,1.9407939771330957E11,2018-01-06
92048.0,94592.85714285714,0.41154099768443975,0.41362754708068344,3213.0,3006.8571428571427,1.0,1.0,34.76045569976926,35.43632890221951,1.516559284082E12,1.5166392739931428E12,1.0,1.0,1.9414001001733582E11,1.9408805661388477E11,2018-01-07
125830.0,106785.28571428571,0.4201345578340008,0.415209057962505,3213.0,3052.4285714285716,1.0,1.0,35.07823150426221,35.38165185261813,1.515624167162E12,1.5166693795671428E12,1.0,1.0,1.941683368444063E11,1.9411073421019586E11,2018-01-08
130842.0,111032.71428571429,0.4178090741861267,0.41530059335951713,3182.0,3092.4285714285716,1.0,1.0,35.229475043628554,35.32499286341081,1.51694704972E12,1.5165902884315715E12,1.0,1.0,1.9419991070289328E11,1.941308993442253E11,2018-01-09
123299.0,111233.28571428571,0.41779439537765384,0.41548274773844573,2940.0,3099.0,1.0,1.0,35.353376243909736,35.27120211299367,1.517503912598E12,1.5168386688472856E12,1.0,1.0,1.9421419020579358E11,1.9414957882455957E11,2018-01-10


## Preprocessing Data

In [7]:
X_data = spark_df.toPandas().drop("date", 1)
X_data.head(10)

,n_rows,moving_avg_7d_n_rows,null_values_ratio,moving_avg_7d_null_values_ratio,max_string_length,moving_avg_7d_max_string_length,min_string_length,moving_avg_7d_min_string_length,avg_string_length,moving_avg_7d_avg_string_length,max_numeric,moving_avg_7d_max_numeric,min_numeric,moving_avg_7d_min_numeric,avg_numeric,moving_avg_7d_avg_numeric
0,40483.0,40483.000000,0.409064,0.409064,2894.0,2894.000000,1.0,1.0,35.460971,35.460971,1.515413e+12,1.515413e+12,1.0,1.0,1.940096e+11,1.940096e+11
1,101110.0,70796.500000,0.417168,0.413116,2902.0,2898.000000,1.0,1.0,35.626088,35.543529,1.517501e+12,1.516457e+12,1.0,1.0,1.940588e+11,1.940342e+11
2,121895.0,87829.333333,0.416519,0.414251,2894.0,2896.666667,1.0,1.0,35.729911,35.605657,1.515765e+12,1.516226e+12,1.0,1.0,1.940834e+11,1.940506e+11
3,115572.0,94765.000000,0.417875,0.415157,3152.0,2960.500000,1.0,1.0,35.645096,35.615517,1.515268e+12,1.515987e+12,1.0,1.0,1.940930e+11,1.940612e+11
4,106311.0,97074.200000,0.416853,0.415496,3149.0,2998.200000,1.0,1.0,35.855670,35.663547,1.519491e+12,1.516688e+12,1.0,1.0,1.941130e+11,1.940716e+11
5,84731.0,95017.000000,0.406372,0.413975,2844.0,2972.500000,1.0,1.0,34.976110,35.548974,1.516477e+12,1.516653e+12,1.0,1.0,1.941186e+11,1.940794e+11
6,92048.0,94592.857143,0.411541,0.413628,3213.0,3006.857143,1.0,1.0,34.760456,35.436329,1.516559e+12,1.516639e+12,1.0,1.0,1.941400e+11,1.940881e+11
7,125830.0,106785.285714,0.420135,0.415209,3213.0,3052.428571,1.0,1.0,35.078232,35.381652,1.515624e+12,1.516669e+12,1.0,1.0,1.941683e+11,1.941107e+11
8,130842.0,111032.714286,0.417809,0.415301,3182.0,3092.428571,1.0,1.0,35.229475,35.324993,1.516947e+12,1.516590e+12,1.0,1.0,1.941999e+11,1.941309e+11
9,123299.0,111233.285714,0.417794,0.415483,2940.0,3099.000000,1.0,1.0,35.353376,35.271202,1.517504e+12,1.516839e+12,1.0,1.0,1.942142e+11,1.941496e+11


In [8]:
shape = np.shape(X_data)
print("X_data shape = {}".format(shape))

X_data shape = (713, 16)

## Model Builder

In [10]:
def autoencoder_builder(
    hidden_layers, dropout_rate, l2_regularizer, epochs, batch_size,
):
    return AutoEncoder(
        hidden_neurons=hidden_layers,
        dropout_rate=dropout_rate,
        l2_regularizer=l2_regularizer,

        hidden_activation='relu',
        output_activation='sigmoid',
        optimizer='adam',
        loss=mean_squared_error,
        
        epochs=epochs,
        batch_size=batch_size,
        
        validation_size=0,
        preprocessing=True
    )


def build_models(builder_method, *args_tests):
    """
    Build models with all the combinations of the args_test passed
    """
    builder_args_names = list(inspect.signature(builder_method).parameters.keys())
    return [
        dict((("model", builder_method(*args)),) + tuple(zip(builder_args_names, args)))
        for args in itertools.product(*args_tests)
    ]


## Models Definition

In [12]:
# Parameters definition
input_dim = shape[1]

hidden_layers_tests = [
    [input_dim, 4, 2, 4, input_dim],
    [input_dim, 8, 4, 2, 4, 8, input_dim],
    [input_dim, 8, 4, 8, input_dim],
    [input_dim, 12, 8, 4, 8, 12, input_dim],
    [input_dim, 10, 6, 10, input_dim],
    [input_dim, 14, 10, 6, 10, 14, input_dim],
    [input_dim, 10, 8, 10, input_dim],
    [input_dim, 14, 10, 8, 10, 14, input_dim],
]
dropout_rate_tests = [0.2, 0.1, 0.05]
l2_regularizer_tests = [0.1, 0.05, 0.01]

# fixed
epochs = [200]
batch_size = [25]

In [13]:
# building models with all the combinations of the test parameters
models = build_models(
    autoencoder_builder,
    hidden_layers_tests,
    dropout_rate_tests,
    l2_regularizer_tests,
    epochs,
    batch_size,
)
models

Out[8]: [{'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.2, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.1,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.2,
 'l2_regularizer': 0.1,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.2, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.05,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.2,
 'l2_regularizer': 0.05,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.2, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.01,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.2,
 'l2_regularizer': 0.01,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.1, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.1,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.1,
 'l2_regularizer': 0.1,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.1, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.05,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.1,
 'l2_regularizer': 0.05,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.1, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.01,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.1,
 'l2_regularizer': 0.01,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.05, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.1,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.05,
 'l2_regularizer': 0.1,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.05, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.05,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preprocessing=True,
 random_state=None, validation_size=0, verbose=1),
 'hidden_layers': [16, 4, 2, 4, 16],
 'dropout_rate': 0.05,
 'l2_regularizer': 0.05,
 'epochs': 200,
 'batch_size': 25},
 {'model': AutoEncoder(batch_size=25, contamination=0.1, dropout_rate=0.05, epochs=200,
 hidden_activation='relu', hidden_neurons=[16, 4, 2, 4, 16],
 l2_regularizer=0.01,
 loss=<function mean_squared_error at 0x7f5dcca41d08>,
 optimizer='adam', output_activation='sigmoid', preproce

In [14]:
# models = random.sample(models, 10) # just for test
# len(models)

## Models K-fold evaluation

In [16]:
def k_fold_evaluate_params(models, k=5):
    kfold = StratifiedKFold(n_splits=k, shuffle=True)
    for model in models:
        k_fold_val_losses = []
        temp_model = autoencoder_builder(
            model["hidden_layers"],
            model["dropout_rate"],
            model["l2_regularizer"],
            model["epochs"],
            model["batch_size"],
        )

        for i, indexes in enumerate(kfold.split(X_data, np.zeros(np.shape(X_data)[0]))):
            train_index, test_index = indexes
            print("Fold:", i)

            temp_model.fit(X_data.iloc[train_index])

            val_loss = temp_model.model_.evaluate(
                temp_model.scaler_.transform(X_data.iloc[test_index]),
                temp_model.scaler_.transform(X_data.iloc[test_index]),
                verbose=False,
            )

            print("val_loss = ", val_loss)
            k_fold_val_losses.append(val_loss)

        model["k_fold_evaluation_loss"] = statistics.mean(k_fold_val_losses)

In [17]:
k_fold_evaluate_params(models)

Fold: 0
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

Model: "sequential_1"
_________________________________________________________________
Layer (type) Output Shape Param # 
=================================================================
dense_1 (Dense) (None, 16) 272 
_________________________________________________________________
dropout_1 (Dropout) (None, 16) 0 
_________________________________________________________________
dense_2 (Dense) (None, 16) 272 
_________________________________________________________________
dropout_2 (Dropout) (None, 16) 0 
_________________________________________________________________
dense_3 (Dense) (None, 16) 272 
_________________________________________________________________
dropout_3 (Dropout) (None, 16) 0 
_________________________________________________________________
dense_4 (Dense) (None, 4) 68 
_________________________________________________________________
dropout_4 (Dropout) (None, 4) 0 
_________________________________________________________________
dense_5 (Dense) (None, 2) 10 
_________________________________________________________________
dropout_5 (Dropout) (None, 2) 0 
_________________________________________________________________
dense_6 (Dense) (None, 4) 12 
_________________________________________________________________
dropout_6 (Dropout) (None, 4) 0 
_________________________________________________________________
dense_7 (Dense) (None, 16) 80 
_________________________________________________________________
dropout_7 (Dropout) (None, 16) 0 
_________________________________________________________________
dense_8 (Dense) (None, 16) 272 
=================================================================
Total params: 1,258
Trainable params: 1,258
Non-trainable params: 0
_________________________________________________________________
None
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1020: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

Epoch 1/200
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:216: The name tf.is_variable_initialized is deprecated. Please use tf.compat.v1.is_variable_initialized instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:223: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer inste

### Top Models in K-fold evaluation

In [19]:
# Sorting by test dataset accuracy, from higher to lower
sorted_models = sorted(models, key=lambda d: d['k_fold_evaluation_loss'], reverse=False)
for i, model in enumerate(sorted_models):
    print("TOP {}:".format(i + 1))
    print("    k_fold_evaluation_loss: {}".format(model["k_fold_evaluation_loss"]))
    print("    hidden_layers: {}".format(model["hidden_layers"]))
    print("    dropout_rate: {}".format(model["dropout_rate"]))
    print("    l2_regularizer: {}".format(model["l2_regularizer"]))

In [20]:
""" Driver Logs:
TOP 1:
    k_fold_evaluation_loss: 1.0590080043359142
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 2:
    k_fold_evaluation_loss: 1.0690934130866738
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 3:
    k_fold_evaluation_loss: 1.0735523109916019
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 4:
    k_fold_evaluation_loss: 1.074629097180708
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 5:
    k_fold_evaluation_loss: 1.079910468245027
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 6:
    k_fold_evaluation_loss: 1.0850195184689044
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 7:
    k_fold_evaluation_loss: 1.08704022145748
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 8:
    k_fold_evaluation_loss: 1.0991598968645346
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 9:
    k_fold_evaluation_loss: 1.10290665576508
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 10:
    k_fold_evaluation_loss: 1.1045440495900658
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 11:
    k_fold_evaluation_loss: 1.1055566493365654
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 12:
    k_fold_evaluation_loss: 1.1062069749616112
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 13:
    k_fold_evaluation_loss: 1.1069269052670059
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 14:
    k_fold_evaluation_loss: 1.1166288161270725
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 15:
    k_fold_evaluation_loss: 1.1174384173061958
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 16:
    k_fold_evaluation_loss: 1.1224442968389194
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 17:
    k_fold_evaluation_loss: 1.1402919983495154
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 18:
    k_fold_evaluation_loss: 1.146943621468476
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 19:
    k_fold_evaluation_loss: 1.1501597796004945
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 20:
    k_fold_evaluation_loss: 1.1513689153854052
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.01
TOP 21:
    k_fold_evaluation_loss: 1.1990468133344776
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 22:
    k_fold_evaluation_loss: 1.2046935412327926
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 23:
    k_fold_evaluation_loss: 1.207477558562054
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 24:
    k_fold_evaluation_loss: 1.218792428099603
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 25:
    k_fold_evaluation_loss: 1.2254141941312897
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 26:
    k_fold_evaluation_loss: 1.2311586133649193
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 27:
    k_fold_evaluation_loss: 1.2362465394537732
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 28:
    k_fold_evaluation_loss: 1.242455088767021
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 29:
    k_fold_evaluation_loss: 1.2468134851276282
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 30:
    k_fold_evaluation_loss: 1.2493799875255267
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 31:
    k_fold_evaluation_loss: 1.2496087040370947
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 32:
    k_fold_evaluation_loss: 1.2511906564781656
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 33:
    k_fold_evaluation_loss: 1.2571724033961562
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 34:
    k_fold_evaluation_loss: 1.2597280617783155
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 35:
    k_fold_evaluation_loss: 1.2654452258138884
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 36:
    k_fold_evaluation_loss: 1.2679727939365064
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 37:
    k_fold_evaluation_loss: 1.2693272276956977
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 38:
    k_fold_evaluation_loss: 1.2990291490163122
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 39:
    k_fold_evaluation_loss: 1.3099610784682432
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.05
TOP 40:
    k_fold_evaluation_loss: 1.3530597979900556
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 41:
    k_fold_evaluation_loss: 1.3538966832343644
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 42:
    k_fold_evaluation_loss: 1.3669852522762023
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 43:
    k_fold_evaluation_loss: 1.3724474110368976
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 44:
    k_fold_evaluation_loss: 1.3731773862366277
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.01
TOP 45:
    k_fold_evaluation_loss: 1.3780428058488445
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 46:
    k_fold_evaluation_loss: 1.3869451707071427
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 47:
    k_fold_evaluation_loss: 1.389986402389863
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 48:
    k_fold_evaluation_loss: 1.395846041235374
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 49:
    k_fold_evaluation_loss: 1.3989843046936417
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 50:
    k_fold_evaluation_loss: 1.4107720441164482
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 51:
    k_fold_evaluation_loss: 1.4124347402158774
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 52:
    k_fold_evaluation_loss: 1.4152178131816635
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 53:
    k_fold_evaluation_loss: 1.4211056297593054
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 54:
    k_fold_evaluation_loss: 1.4325671763181993
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.1
TOP 55:
    k_fold_evaluation_loss: 1.43833898363073
    hidden_layers: [16, 12, 8, 4, 8, 12, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 56:
    k_fold_evaluation_loss: 1.4467190403333852
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 57:
    k_fold_evaluation_loss: 1.447042263952556
    hidden_layers: [16, 4, 2, 4, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 58:
    k_fold_evaluation_loss: 1.4624479563520612
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 59:
    k_fold_evaluation_loss: 1.4742698092983355
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 60:
    k_fold_evaluation_loss: 1.4993805084042533
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.1
    l2_regularizer: 0.1
TOP 61:
    k_fold_evaluation_loss: 1.7366891037357577
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 62:
    k_fold_evaluation_loss: 2.0330188727820317
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.01
TOP 63:
    k_fold_evaluation_loss: 2.048950632806953
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 64:
    k_fold_evaluation_loss: 2.054231382038609
    hidden_layers: [16, 16, 12, 8, 12, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 65:
    k_fold_evaluation_loss: 2.056917084419914
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 66:
    k_fold_evaluation_loss: 2.3589172804330967
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 67:
    k_fold_evaluation_loss: 2.520508529740015
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 68:
    k_fold_evaluation_loss: 2.617198834761626
    hidden_layers: [16, 14, 10, 6, 10, 14, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.05
TOP 69:
    k_fold_evaluation_loss: 2.9319758413507944
    hidden_layers: [16, 16, 8, 16, 16]
    dropout_rate: 0.2
    l2_regularizer: 0.05
TOP 70:
    k_fold_evaluation_loss: 3.499163974262536
    hidden_layers: [16, 8, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 71:
    k_fold_evaluation_loss: 3.760863760115771
    hidden_layers: [16, 8, 4, 2, 4, 8, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
TOP 72:
    k_fold_evaluation_loss: 4.3181867467116035
    hidden_layers: [16, 10, 6, 10, 16]
    dropout_rate: 0.05
    l2_regularizer: 0.1
"""